# AI Fall 00 - Hands-on - Game

<div style="font-size: 16px">
<b>Paria Khoshtab 810198387</b>
<hr>
</div>

## Goal

This project aims to get more familiar with `adversarial game trees` and `min-max algorithm`, which are used in the implementation of most `multi-agent` games. Also, we examine the impact of tree depth on agents' decisions.

## Brief Description

In this problem, we have been given incomplete codes of the `Checkers` game, which involves diagonal moves of uniform game pieces and mandatory captures by jumping over opponent pieces. The goal of Checkers is to remove all your opponent's pieces from the board. To capture an opponent's piece and remove it from the board, you need to jump over their piece with one of yours. If, after jumping, your piece could make another jump, it is allowed to make an additional jump, which is called "double-jumping." If one of your pieces gets to the opposite side of the board, 
it will turn into a King. Kings can move and jump diagonally in any direction. You win by removing your opponent's pieces from the board or if your opponent can't make a move.<br> What we need to do here is complete the incomplete codes that are
related to the minimax algorithm.

## Modeling the Problem

The state space of this problem consists of states, including information about the board (position of pieces).<br>
- <b>Initial state</b>: initial board game, including 24 pieces<br>
- <b>Players</b>: P = {1, 2}<br>
- <b>Actions</b>: 1) diagonal move 2) removing opponent's pieces from the board by jumping over them<br>
- <b>Terminal state</b>: removing all of opponent's pieces from the board<hr>

## Implementation

First, we have to implement the `getValidMoves` function using the `_traverseLeft` and `_traverseRight` functions.<br>
This function returns all the valid moves that a piece can make.<br>
As we can see below, king pieces can move and jump in 4 directions, but non-king pieces can move and jump in only two directions.

In [ ]:
def getValidMoves(self, piece):
    valid_moves = {}

    if piece.king:
        valid_moves.update(self._traverseLeft(piece.row - 1, max(-1, piece.row - 3), -1, piece.color, piece.col - 1))
        valid_moves.update(self._traverseRight(piece.row - 1, max(-1, piece.row - 3), -1, piece.color, piece.col + 1))
        valid_moves.update(self._traverseLeft(piece.row + 1, min(ROWS, piece.row + 3), +1, piece.color, piece.col - 1))
        valid_moves.update(self._traverseRight(piece.row + 1, min(ROWS, piece.row + 3), +1, piece.color, piece.col + 1))
    elif piece.color == RED:
        valid_moves.update(self._traverseLeft(piece.row - 1, max(-1, piece.row - 3), -1, piece.color, piece.col - 1))
        valid_moves.update(self._traverseRight(piece.row - 1, max(-1, piece.row - 3), -1, piece.color, piece.col + 1))
    elif piece.color == WHITE:
        valid_moves.update(self._traverseLeft(piece.row + 1, min(ROWS, piece.row + 3), +1, piece.color, piece.col - 1))
        valid_moves.update(self._traverseRight(piece.row + 1, min(ROWS, piece.row + 3), +1, piece.color, piece.col + 1))

    return valid_moves

The `simulateMove` function takes a piece, destination position (move), the board, and skipped pieces as parameters in order to
simulate a move of a piece and remove skipped pieces from the board.

In [ ]:
def simulateMove(piece, move, board, skip):
    board.remove(skip)
    board.move(piece, move[0], move[1])
    return board

The `getAllMoves` function takes color and the board as parameters in order to return all the successor states (boards) of the 
current state(board) based on its color, using `getValidMoves` and `simulateMove` functions.

In [ ]:
def getAllMoves(board, color):
    boards = []
    for piece in board.getAllPieces(color):
        for move, skipped in board.getValidMoves(piece).items():
            newBoard = deepcopy(board)
            newBoard = simulateMove(newBoard.getPiece(piece.row, piece.col), move, newBoard, skipped)
            boards.append(newBoard)
    return boards

`minimax` **depth-limited** function returns the next optimal move for a player and the minimax value (evaluation) of the selected state,
assuming that your opponent also plays optimally.<br>
We need to implement a function that estimates the value or goodness of the board depending on the placement of pieces
on the board in minimax, typically a weighted linear sum of features.
This function is often known as the **Evaluation Function**. The ideal evaluation function
returns the actual minimax value of the state.
The evaluation function is unique for every type of game.<br>
We compute minimax value (evaluation) of state "s" as follows:<br>
Terminal states(states with given depth): V(s) = evaluate(s)<br>
States under opponent's control: V(s) = min V(s') ; s' is a member of successors(s)<br>
States under agent's control: V(s) = max V(s') ; s' is a member of successors(s)<br>
Based on the descriptions above, we implement the `minimax` function as follows:

In [ ]:
def minimax(position, depth, maxPlayer):
    if depth == 0:
        return position.evaluate(), position

    value = None
    newBoard = deepcopy(position)

    if maxPlayer == True:
        value = -math.inf
        for board in getAllMoves(position, WHITE):
            newValue = minimax(board, depth - 1, False)[0]
            if newValue >= value:
                value = newValue
                newBoard = deepcopy(board)

    elif maxPlayer == False:
        value = math.inf
        for board in getAllMoves(position, RED):
            newValue = minimax(board, depth - 1, True)[0]
            if newValue <= value:
                value = newValue
                newBoard = deepcopy(board)

    return value, newBoard

## Examining the Algorithm with Different Depths

* Note that the analysis of the below tests is what we expect, 
but in reality, our game gets stuck in a local answer and falls into a loop.

#### Test # 1: Low Depth

If we consider a depth 1 for both players, the algorithm gets very fast at the expense of optimality. 
The algorithm's speed improves a lot, but players do not necessarily make the best decisions by searching the whole tree because they can not predict many moves. So, there is no guarantee that either the RED player or the WHITE player will win the game.

#### Test # 2 High Depth

If we consider depth 5 for the RED player and depth 2 for the WHITE player, the algorithm gets slower than the previous test, but since the RED player examines more depth of the tree, its evaluation function becomes more accurate, resulting in better decisions than the WHITE player takes. So, the RED player is more likely to win the game than the WHITE player.

#### Test # 3 Equal Depth

If we consider depth 5 for both players, the algorithm gets slower than the previous two tests, but since both players 
examine more depth of the tree and can predict more moves, their evaluation functions become more accurate,
resulting in better decisions, so both players make better and more optimal decisions than Test 1 with low depth. 
Thus, the probability of the RED player winning the game increases as the probability of winning the WHITE player increases, 
but I guess the player who starts the game has a better chance of winning.

## Conclusion

The Minimax algorithm takes into account all the possible moves that players can take at any given time during the game.
This enables the algorithm to minimize the opponent’s advantage while simultaneously maximizing the agent’s advantage
at every turn.
Though effective in finding accurate solutions in simple games, the minimax algorithm is not the best choice for
games with exceptionally high branching factors. To improve this drawback, Alpha Beta Pruning is implemented
in the Minimax Algorithm.
Alpha Beta Pruning seeks to decrease the number of nodes that are evaluated by the minimax algorithm
by passing two extra parameters, Alpha and Beta.<br>
Additionally, evaluation functions are always imperfect, but the deeper in the tree the evaluation function is buried,
the less the quality of the evaluation function matters. There is always a trade-off between the optimality of decisions made by
the evaluation function and the cost(time) of the minimax algorithm. By defining a good evaluation function, we make better
decisions in order to get close to the optimal decisions and reach the goal.